# Preparing the IMDb dataset.

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import json
import requests

api_key1 = "b58bb13f"
api_key2 = "9bf3218a"

RANDOM_STATE = 1212

cmu_data = pd.read_csv("cmu_dataset_v1.csv")
cmu_backups = pd.read_csv("cmu_movies_shuffled.csv")
asd = pd.DataFrame(columns=["Title", "Year", "Genre", "Plot"])

In [34]:
# Initialize asd as a DataFrame


movie_name = "Dune"
year = 2024
url = f"http://www.omdbapi.com/?t={movie_name}&y={year}&plot=full&apikey={api_key1}"
response = requests.get(url)
data = response.json()

# Check if the request was successful
if response.status_code == 200:
    asd.loc[len(asd)] = [data['Title'], data['Year'], data['Genre'], data['Plot']]
else:
    print(f"Error: {response.status_code} - {response.text}")

print(data)

Error: 401 - {"Response":"False","Error":"Request limit reached!"}
{'Response': 'False', 'Error': 'Request limit reached!'}


In [24]:
# check what asd is
asd.head()

,Title,Year,Genre,Plot
0,Inception,2010,"Action, Adventure, Sci-Fi","Dom Cobb is a skilled thief, the absolute best..."
1,Interstellar,2014,"Adventure, Drama, Sci-Fi","Earth's future has been riddled by disasters, ..."


In [43]:
# using the cmu_data dataframe to get the movie names and years, make api requests to build a new dataframe with the new movies.
# if {"Response":"False","Error":"Movie not found!"}, the movie was not found and it will not be added to the new dataframe.
# instead, the movie will be added to a df of movies that were not found.
# if the plot summary contains less than 2 sentences, the movie will not be added to the new dataframe.
# use this code to check the length of the plot summary:
# plot_summaries = plot_summaries[plot_summaries["plot"].apply(lambda x: x.count(".") >= 2)]

# loop through the cmu_data dataframe and make api requests to get the movie data

import time

REMEMBER TO CHANGE THESE VARIABLES TO THE CORRECT ONES
not_found = pd.DataFrame(columns=["movie_name", "year"])
imdb_data = pd.DataFrame(columns=["movie_name", "year", "genre", "plot"])

for index, row in cmu_data.iterrows():
    # check if 'movie_name' and 'year' are already in not_found or imdb_data. if they are, skip the row.
    if row["movie_name"] in not_found["movie_name"].values and row["year"] in not_found["year"].values:
        continue
    if row["movie_name"] in imdb_data["movie_name"].values and row["year"] in imdb_data["year"].values:
        continue

    url = f"http://www.omdbapi.com/?t={row['movie_name']}&y={row['year']}&plot=full&apikey={api_key2}"
    response = requests.get(url)
    data = response.json()

    time.sleep(1)

    if data["Response"] == "False":
        if data["Error"] == "Request limit reached!":
            print("Request limit reached!")
            break
        else: 
            not_found.loc[len(not_found)] = [row["movie_name"], row["year"]]
    else:
        if data["Plot"].count(".") >= 2:
            imdb_data.loc[len(imdb_data)] = [data['Title'], data['Year'], data['Genre'], data['Plot']]
        else:
            not_found.loc[len(not_found)] = [row["movie_name"], row["year"]]


# remove all rows in not_found from cmu_data 
# for each row removed from cmu data, add a replacement row from cmu_backups and remove that row from cmu_backups
# if a 'Comedy' movie is removed from cmu_data, the replacement row has to be a 'Comedy' movie


KeyboardInterrupt: 

In [45]:
imdb_data.head()

,movie_name,year,genre,plot
0,The Laurel-Hardy Murder Case,1930,"Short, Comedy, Crime",It looks as if the boys might no longer need t...
1,Kung Phooey!,2003,Comedy,The chosen one from China's mystical Shur-Li T...
2,Things Are Tough All Over,1982,"Action, Comedy",Cheech and Chong work briefly as car-wash atte...
3,The Twelve Chairs,1970,"Comedy, Drama",A treasure hunt. An aging ex-nobleman of the C...
4,Crocodile Dundee II,1988,"Action, Adventure, Comedy",One short year after the life-changing adventu...


In [44]:
imdb_data.shape, not_found.shape

((8, 4), (6, 2))

In [42]:
not_found.to_csv("not_found_movies.csv", index=False)
imdb_data.to_csv("imdb_data_found.csv", index=False)